In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext line_profiler

In [3]:
import itertools
import torch

from tsvar.simulate import GrangerBuscaSimulator
from tsvar.wold_model import VariationalWoldModel

---

## Generate small toy example dataset

Define model parameters

In [4]:
dim = 3

baseline = np.array([0.1, 0.1, 0.2])

# adjacency[i,j] = magnitude of influence from i to j
adjacency = np.array([
    [0.3, 0.8, 0.1],
    [0.2, 0.3, 0.1],
    [0.2, 0.1, 0.0]
])

end_time = 1e5

for i, j in itertools.product(range(dim), repeat=2):
    print(f"w({j:d} -> {i:d}) = {adjacency[j,i]:.2f}")

w(0 -> 0) = 0.30
w(1 -> 0) = 0.20
w(2 -> 0) = 0.20
w(0 -> 1) = 0.80
w(1 -> 1) = 0.30
w(2 -> 1) = 0.10
w(0 -> 2) = 0.10
w(1 -> 2) = 0.10
w(2 -> 2) = 0.00


Simulate a dataset

In [5]:
wold_sim = GrangerBuscaSimulator(mu_rates=baseline, Alpha_ba=adjacency, Beta_b=np.ones((dim,)))
events = wold_sim.simulate(end_time, seed=42)
events = [torch.tensor(ev, dtype=torch.float) for ev in events]


print(list(map(len, events)))
print()
print(events[0])
print()
print(events[1])
print()
print(events[2])

[36304, 55138, 28497]

tensor([2.5502e+00, 3.3542e+00, 9.4456e+00,  ..., 9.9993e+04, 9.9998e+04,
        9.9998e+04])

tensor([1.0916e+01, 1.1661e+01, 1.1825e+01,  ..., 9.9996e+04, 9.9996e+04,
        9.9997e+04])

tensor([5.7092e+00, 2.1376e+01, 2.8914e+01,  ..., 9.9992e+04, 9.9993e+04,
        9.9995e+04])


---

## Test variational inference algorithm

Create model object and set the data.

In [6]:
self = VariationalWoldModel(verbose=True)
self.set_data(events)

Define the parameters of the prior.

In [10]:
as_pr = 1.0 * np.ones((dim + 1, dim))
ar_pr = 1.0 * np.ones((dim + 1, dim))

# lamb = 2.0 * (dim + 1) * np.ones(dim + 1)
lamb = 0.01 * self.n_jumps

zc_pr = [1.0 * np.ones((len(events[i]), dim+1)) for i in range(dim)]

In [12]:
self.fit(as_pr=as_pr, ar_pr=ar_pr, zc_pr=zc_pr, lamb=lamb, max_iter=100, tol=1e-5)

-------------------------------------------------- 0
Alpha posterior shape:
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
Z posterior probabilities
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 ...
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
-------------------------------------------------- 1
[36304 55138 28497]
Alpha posterior shape:
[[ 8977.   13685.5   7025.25]
 [ 8977.   13685.5   7025.25]
 [ 8977.   13685.5   7025.25]
 [ 8977.   13685.5   7025.25]]
0.622597316673121 0.9999989754903778
0.6226428217590246 0.9999989754903779
0.622634018417539 0.9999989754903776
Z posterior probabilities
[[1.      0.      0.      0.     ]
 [1.      0.      0.      0.     ]
 [0.64337 0.35663 0.      0.     ]
 ...
 [0.50994 0.25299 0.10494 0.13213]
 [0.49364 0.15116 0.06928 0.28591]
 [0.47078 0.08323 0.30903 0.13696]]
-------------------------------------------------- 2
[36304 55138 28497]
Alpha posterior shape:
[[16617.31958 25005.10849 13138.31107]

/Users/trouleau/Workspace/EPFL/research/ongoing/var-wold/tsvar/wold_model.py:238: RuntimeWarning: divide by zero encountered in log
  - digamma(Fi_k[:, np.newaxis]) - np.log(C_ik[i][:, np.newaxis]))


0.7859510998361572 0.9999943750873567
0.7823714376032459 0.999994623682633
0.7889537580327048 0.9999941699199634
Z posterior probabilities
[[1.      0.      0.      0.     ]
 [1.      0.      0.      0.     ]
 [0.82759 0.17241 0.      0.     ]
 ...
 [0.73213 0.13652 0.06778 0.06357]
 [0.72869 0.08387 0.04601 0.14144]
 [0.68529 0.04554 0.20236 0.06681]]
-------------------------------------------------- 3
[36304 55138 28497]
Alpha posterior shape:
[[24540.66981 36771.59692 19456.65637]
 [ 3531.08309  5846.11226  2683.85848]
 [ 5035.36915  7945.33498  3834.68273]
 [ 2800.87795  4178.95584  2125.80242]]
0.8939721157857707 0.9999728677293767
0.8891567888124464 0.9999749403979447
0.8976673824876783 0.9999711758114064
Z posterior probabilities
[[1.      0.      0.      0.     ]
 [1.      0.      0.      0.     ]
 [0.92614 0.07386 0.      0.     ]
 ...
 [0.87476 0.06244 0.03693 0.02587]
 [0.87801 0.03868 0.02528 0.05803]
 [0.83801 0.02132 0.11286 0.02782]]
------------------------------------

RuntimeError: Negative alpha!

In [ ]:
self.delta_ikj[0].shape

In [ ]:
self.valid_mask_ikj[0]